In [1]:
%matplotlib inline
# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
vacation_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
vacation_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Mountain Home,US,33.80,scattered clouds,43.13,-115.69,Mountain Home Inn
1,1,Jamestown,AU,75.81,broken clouds,-33.21,138.60,Jamestown Motel Terraces
2,2,Touros,BR,78.80,scattered clouds,-5.20,-35.46,INN NEW HORIZON
3,5,Vostok,RU,10.69,few clouds,46.45,135.83,NaN
4,6,Yellowknife,CA,-18.00,clear sky,62.45,-114.38,The Explorer Hotel
...,...,...,...,...,...,...,...,...
477,559,Anamur,TR,51.80,thunderstorm with light rain,36.08,32.84,Luna Piena Beach Hotel
478,560,Chumikan,RU,2.19,clear sky,54.72,135.31,Fortuna
479,561,Mogadishu,SO,80.60,broken clouds,2.04,45.34,Olympic Hotel
480,563,Sayyan,YE,61.50,clear sky,15.17,44.32,الاستاذة مروى اليوسفي


In [26]:
#split cities into 4 separate dataframes

city1_df = vacation_df.loc[(vacation_df["City"] == 'Campbellsville')]
city2_df = vacation_df.loc[(vacation_df["City"] == 'Lander')]
city3_df = vacation_df.loc[(vacation_df["City"] == 'Redmond')]
city4_df = vacation_df.loc[(vacation_df["City"] == 'Saint Anthony')]
city1_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
179,212,Campbellsville,US,44.6,clear sky,37.34,-85.34,Holiday Inn Express Campbellsville


In [43]:
# pack tuples for coordinates
city1_tuple = (city1_df['Lat'].values[0], city1_df['Lng'].values[0])
city2_tuple = (city2_df['Lat'].values[0], city2_df['Lng'].values[0])
city3_tuple = (city3_df['Lat'].values[0], city3_df['Lng'].values[0])
city4_tuple = (city4_df['Lat'].values[0], city4_df['Lng'].values[0])

# plot directions map
fig = gmaps.figure()
full_route = gmaps.directions_layer(
        city1_tuple, city4_tuple, waypoints=[city2_tuple, city3_tuple],
        travel_mode='DRIVING')
fig.add_layer(full_route)
fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# new dataframe from all the destinations
full_route_df = vacation_df.loc[(vacation_df["City"] == 'Campbellsville') | 
                                (vacation_df["City"] == 'Lander') |
                                (vacation_df["City"] == 'Redmond') |
                                (vacation_df["City"] == 'Saint Anthony')]
full_route_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
179,212,Campbellsville,US,44.6,clear sky,37.34,-85.34,Holiday Inn Express Campbellsville
183,218,Lander,US,25.0,clear sky,42.83,-108.73,Holiday Inn Express & Suites Lander
210,249,Redmond,US,44.6,light rain,47.67,-122.12,Seattle Marriott Redmond
215,256,Saint Anthony,US,26.6,overcast clouds,43.97,-111.68,Hunter's Escape


In [62]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} with a maximum temperature of {Max Temp}°F</dd>
</dl>
"""

In [63]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in full_route_df.iterrows()]

In [64]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = full_route_df[["Lat", "Lng"]]
max_temp = full_route_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))